In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# driver.find_element(By.CSS_SELECTOR, "table.type2")

In [225]:
driver = uc.Chrome()

In [226]:
driver.get('https://new.land.naver.com/complexes/166857?ms=35.8489594,128.6091271,15&a=ABYG:PRE&b=B1&e=RETAIL&ad=true')

#### 테스트


In [4]:
# 1. 중복된 상품 없이 100개 이상의 상품 정보 수집하기(제목,가격,평점)
# 2. 수집한 정보는 데이터 프레임으로 만들기

# 스크롤하려는 div 요소 선택
div_element = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')

while True:
     # JavaScript를 사용하여 div 내부에서 스크롤
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', div_element)
    # print(body) 잘가져오는지 확인
    #scroll.send_keys(Keys.PAGE_DOWN) # 페이지 아래로 휠 
    time.sleep(2)

    html = BeautifulSoup(driver.page_source)
    

KeyboardInterrupt: 

In [28]:
<span class="text">대구범어아이파크 101동</span>
<span class="type">전세</span>
<span class="price">8억</span>
<span class="spec">148/121m², 11/23층, 남향</span>

SyntaxError: invalid decimal literal (1780689919.py, line 4)

#### 스크래핑

In [217]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import time

# 상품 정보 추출 함수
def extract_listings(driver, target_count=10):
    box = []

    # 스크롤하려는 div 요소 선택
    div_element = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')

    # tqdm을 사용해 진행 상황 표시
    with tqdm(total=target_count, desc="Scraping listings") as pbar:
        while len(box) < target_count:
            # JavaScript를 사용하여 div 내부에서 스크롤
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', div_element)
            time.sleep(1)

            html = BeautifulSoup(driver.page_source, 'html.parser')

            # 상품 정보 추출
            listings = html.find('div', class_='item_area').find_all("div", class_="item false")  # 클래스명 수정 필요

            for listing in listings:
                title_tag = listing.find('span', class_='text')  # 상품 이름
                type_tag = listing.find('span', class_='type')    # 상품 유형
                price_tag = listing.find('span', class_='price')   # 가격
                spec_tag = listing.find('span', class_='spec')     # 상세 정보 (평수, 층, 방향 등)

                if title_tag and type_tag and price_tag and spec_tag:
                    # 각 태그에서 텍스트 추출
                    title = title_tag.get_text(strip=True)
                    type_ = type_tag.get_text(strip=True)
                    price = price_tag.get_text(strip=True)

                    # spec 태그에서 평수, 층, 방향 분리 (', '로 분리)
                    spec_text = spec_tag.get_text(strip=True)
                    spec_parts = spec_text.split(', ')
                    area = spec_parts[0]  # 평수
                    floor = spec_parts[1]  # 층
                    direction = spec_parts[2] if len(spec_parts) > 2 else ''  # 방향 (optional)

                    # 새로운 상품 정보를 사전으로 생성
                    new_entry = {
                        '아파트 이름': title,
                        '유형': type_,
                        '가격': price,
                        '평수': area,
                        '층': floor,
                        '방향': direction
                    }

                    # 중복 확인: 새로운 데이터가 이미 box에 있는지 확인
                    if not any((new_entry['아파트 이름'] == item['아파트 이름'] and
                                new_entry['유형'] == item['유형'] and
                                new_entry['가격'] == item['가격'] and
                                new_entry['평수'] == item['평수'] and
                                new_entry['층'] == item['층'] and
                                new_entry['방향'] == item['방향']) for item in box):
                        # 중복이 아니라면 저장
                        box.append(new_entry)

                        # tqdm 진행 상황 업데이트
                        pbar.update(1)

                    # 타겟 개수에 도달하면 중단
                    if len(box) >= target_count:
                        break

            # # 스크롤 내려 더 많은 데이터 로드
            # driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # time.sleep(2)  # 페이지 로딩 대기

    # 수집한 데이터를 DataFrame으로 변환하여 반환
    return pd.DataFrame(box)


In [219]:
# 예시 사용법
# driver.get("https://example.com")  # 크롤링할 웹 페이지 URL 입력
# time.sleep(5)  # 페이지 로딩 대기

listings = extract_listings(driver, target_count=118)

# for listing in listings:
#     print(listing)

listings

Scraping listings: 100%|█████████████████████████████████████████████████████████████| 118/118 [00:01<00:00, 89.67it/s]


,아파트 이름,유형,가격,평수,층,방향
0,대구범어아이파크 103동,월세,"5,000/150",109B/84m²,6/23층,남서향
1,대구범어아이파크 101동,월세,3억/250,148/121m²,중/23층,남향
2,대구범어아이파크 101동,전세,"7억 5,000",148/121m²,고/23층,남향
3,대구범어아이파크 103동,전세,"4억 5,000",109B/84m²,7/23층,남서향
4,대구범어아이파크 101동,전세,8억,148/121m²,16/23층,남향
...,...,...,...,...,...,...
113,대구범어아이파크 102동,전세,5억,109B/84m²,15/22층,남서향
114,대구범어아이파크 104동,월세,2억/130,109B/84m²,중/25층,남서향
115,대구범어아이파크 101동,전세,"4억 4,000",110C/84m²,중/23층,동향
116,대구범어아이파크 102동,월세,"5,000/170",109B/84m²,중/22층,남서향


In [190]:
listings.to_excel('APT.xlsx', index=False)


In [209]:
# 특정 열 기준으로 중복 행 확인
duplicates = listings[listings.duplicated(subset=['아파트 이름', '유형', '가격','평수','층','방향'])]

# 중복된 행이 있는지 확인
if not duplicates.empty:
    print("중복된 행이 발견되었습니다:")
    print(duplicates)
else:
    print("중복된 행이 없습니다.")


중복된 행이 없습니다.


In [237]:
# 첫번째 크롤링 페이지
driver = uc.Chrome()
driver.get('https://new.land.naver.com/complexes/166857?ms=35.8489594,128.6091271,15&a=ABYG:PRE&b=B1&e=RETAIL&ad=true')

In [241]:

  # 페이지 로딩 대기
listings1 = extract_listings(driver, target_count=118)  # 첫 번째 링크에서 정보 수집

# 지역 컬럼 추가
listings1['지역'] = '대구광역시 수성구 범어동'




Scraping listings: 100%|█████████████████████████████████████████████████████████████| 118/118 [01:17<00:00,  1.53it/s]


In [243]:
# # 두 번째 URL에서 상품 정보 수집
driver = uc.Chrome()
driver.get("https://new.land.naver.com/complexes/145541?ms=35.8652902,128.6329665,15&a=ABYG:PRE&b=B1&e=RETAIL&ad=true")  # 두 번째 크롤링할 웹 페이지 URL 입력

In [249]:

listings2 = extract_listings(driver, target_count=39)  # 두 번째 링크에서 정보 수집

# 지역 컬럼 추가
listings2['지역'] = '대구광역시 수성구 범어동'




Scraping listings: 100%|███████████████████████████████████████████████████████████████| 39/39 [00:01<00:00, 32.87it/s]


In [251]:
# # 세 번째 URL에서 상품 정보 수집
driver = uc.Chrome()
driver.get("https://new.land.naver.com/complexes/140871?ms=35.8606731,128.6303915,15&a=ABYG:PRE&b=B1&e=RETAIL&ad=true")  # 세 번째 크롤링할 웹 페이지 URL 입력


In [253]:

  # 페이지 로딩 대기
listings3 = extract_listings(driver, target_count=100)  # 두 번째 링크에서 정보 수집

# 지역 컬럼 추가
listings3['지역'] = '대구광역시 수성구 만천동'




Scraping listings: 100%|█████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 74.90it/s]


In [259]:
import pandas as pd

# 데이터프레임 concat
combined_listings = pd.concat([listings1, listings2, listings3], ignore_index=True)

# 결과 확인
combined_listings

,아파트 이름,유형,가격,평수,층,방향,지역
0,대구범어아이파크 103동,월세,"5,000/150",109B/84m²,6/23층,남서향,대구광역시 수성구 범어동
1,대구범어아이파크 101동,전세,"7억 5,000",148/121m²,고/23층,남향,대구광역시 수성구 범어동
2,대구범어아이파크 101동,월세,3억/250,148/121m²,중/23층,남향,대구광역시 수성구 범어동
3,대구범어아이파크 103동,전세,"4억 5,000",109B/84m²,7/23층,남서향,대구광역시 수성구 범어동
4,대구범어아이파크 101동,전세,8억,148/121m²,16/23층,남향,대구광역시 수성구 범어동
...,...,...,...,...,...,...,...
252,힐스테이트만촌역(주상복합) 104동,전세,5억,116B/84m²,중/29층,남동향,대구광역시 수성구 만천동
253,힐스테이트만촌역(주상복합) 105동,전세,5억,116B/84m²,19/29층,남동향,대구광역시 수성구 만천동
254,힐스테이트만촌역(주상복합) 102동,매매,"13억 5,007",174A/136m²,저/31층,남서향,대구광역시 수성구 만천동
255,힐스테이트만촌역(주상복합) 104동,매매,"9억 3,310",97/65m²,고/29층,동향,대구광역시 수성구 만천동


In [261]:
# 데이터프레임을 엑셀 파일로 저장
combined_listings.to_excel("APT_매물.xlsx", index=False)

In [109]:
import undetected_chromedriver as uc  # undetected_chromedriver 임포트
from bs4 import BeautifulSoup
import time

# 키워드 설정
keyword = "서울시 강서구"

# Chrome 드라이버 실행
driver = uc.Chrome()

# URL 구성 및 페이지 열기
url = "https://m.land.naver.com/search/result/{}".format(keyword)
driver.get(url)

# 페이지가 로드될 시간을 기다립니다. (필요에 따라 조정)
time.sleep(3)  # 3초 대기

# 페이지 소스 가져오기
html = driver.page_source

# BeautifulSoup으로 HTML 파싱
soup = BeautifulSoup(html, "lxml")

In [111]:
#응답 메시지 속에서 원하는 데이터 얻기
#  filter: {
#             lat: '37.550985',
#             lon: '126.849534',
#             z: '12',
#             cortarNo: '1150000000',
#             cortarNm: '강서구',
#             rletTpCds: '*',
#             tradTpCds: 'A1:B1:B2'
#         },

value = soup.split("filter: {")[1].split("}")[0].replace(" ","").replace("'","")

lat = value.split("lat:")[1].split(",")[0]
lon = value.split("lon:")[1].split(",")[0]
z = value.split("z:")[1].split(",")[0]
cortarNo = value.split("cortarNo:")[1].split(",")[0]
rletTpCds = value.split("rletTpCds:")[1].split(",")[0]
tradTpCds = value.split("tradTpCds:")[1].split()[0]

# lat - btm : 37.550985 - 37.4331698 = 0.1178152
# top - lat : 37.6686142 - 37.550985 = 0.1176292
lat_margin = 0.118

# lon - lft : 126.849534 - 126.7389841 = 0.1105499
# rgt - lon : 126.9600839 - 126.849534 = 0.1105499
lon_margin = 0.111

btm=float(lat)-lat_margin
lft=float(lon)-lon_margin
top=float(lat)+lat_margin
rgt=float(lon)+lon_margin

# 최초 요청 시 디폴트 값으로 설정되어 있으나, 원하는 값으로 구성
rletTpCds="APT" # 아파트
tradTpCds="A1:B1:B2" #매매/전세/월세 매물 확인


TypeError: 'NoneType' object is not callable

In [107]:
print(soup)  # soup 내용 출력하여 filter: { 가 포함되어 있는지 확인


<html data-user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36" lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="No-Cache" http-equiv="Cache-Control"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="index,nofollow" name="robots"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" id="_viewport" name="viewport"/>
<meta contents="always" name="referrer"/>
<title>네이버페이 부동산</title>
<link href="https://fin.land.naver.com/_front/image/apple-touch-icon.png" rel="apple-touch-icon" sizes="192x192"/>
<lin